<a href="https://colab.research.google.com/github/dvschultz/stylegan2-training/blob/main/SG2_ADA_Generate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 1.x
!nvidia-smi

TensorFlow 1.x selected.
Fri Oct 23 15:40:37 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                               

In [2]:
!git clone https://github.com/dvschultz/stylegan2-ada
!pip install opensimplex

Cloning into 'stylegan2-ada'...
remote: Enumerating objects: 59, done.
remote: Counting objects: 100% (59/59), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 135 (delta 32), reused 29 (delta 14), pack-reused 76
Receiving objects: 100% (135/135), 1.15 MiB | 8.88 MiB/s, done.
Resolving deltas: 100% (57/57), done.


In [3]:
%cd stylegan2-ada

/content/stylegan2-ada


In [4]:
!gdown --id 1M30OlXUDslaRkNE3gVg8eX8ODM4UOVdf -O /content/ladiescrop.pkl

Downloading...
From: https://drive.google.com/uc?id=1M30OlXUDslaRkNE3gVg8eX8ODM4UOVdf
To: /content/ladiescrop.pkl
382MB [00:03, 99.8MB/s]


In [5]:
!python generate.py generate-images --network="/content/ladiescrop.pkl" --seeds=0-10 --outdir="./out/"

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)1531904/45929032 bytes (3.3%)4702208/45929032 bytes (10.2%)8708096/45929032 bytes (19.0%)12869632/45929032 bytes (28.0%)17006592/45929032 bytes (37.0%)21069824/45929032 bytes (45.9%)25255936/45929032 bytes (55.0%)29483008/45929032 bytes (64.2%)33742848/45929032 bytes (73.5%)37937152/45929032 bytes (82.6%)41943040/45929032 bytes (91.3%)45929032/45929032 bytes (100.0%)
  Done
File saved as /root/.

In [ ]:
import pickle
import argparse
import numpy as np
import PIL.Image
import dnnlib
import dnnlib.tflib as tflib
import re
import sys
from tqdm import tqdm_notebook as tqdm
import scipy.interpolate as interpolate
from opensimplex import OpenSimplex
import os

def generate_images(network_pkl, seeds, truncation_psi, outdir, class_idx, dlatents_npz):
    tflib.init_tf()
    print('Loading networks from "%s"...' % network_pkl)
    with dnnlib.util.open_url(network_pkl) as fp:
        _G, _D, Gs = pickle.load(fp)

    os.makedirs(outdir, exist_ok=True)

    # Render images for a given dlatent vector.
    if dlatents_npz is not None:
        print(f'Generating images from dlatents file "{dlatents_npz}"')
        dlatents = np.load(dlatents_npz)['dlatents']
        assert dlatents.shape[1:] == (18, 512) # [N, 18, 512]
        imgs = Gs.components.synthesis.run(dlatents, output_transform=dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True))
        for i, img in enumerate(imgs):
            fname = f'{outdir}/dlatent{i:02d}.png'
            print (f'Saved {fname}')
            PIL.Image.fromarray(img, 'RGB').save(fname)
        return

    # Render images for dlatents initialized from random seeds.
    Gs_kwargs = {
        'output_transform': dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True),
        'randomize_noise': False
    }
    if truncation_psi is not None:
        Gs_kwargs['truncation_psi'] = truncation_psi

    noise_vars = [var for name, var in Gs.components.synthesis.vars.items() if name.startswith('noise')]
    label = np.zeros([1] + Gs.input_shapes[1][1:])
    if class_idx is not None:
        label[:, class_idx] = 1

    for seed_idx, seed in enumerate(seeds):
        print('Generating image for seed %d (%d/%d) ...' % (seed, seed_idx, len(seeds)))
        rnd = np.random.RandomState(seed)
        z = rnd.randn(1, *Gs.input_shape[1:]) # [minibatch, component]
        tflib.set_vars({var: rnd.randn(*var.shape.as_list()) for var in noise_vars}) # [height, width]
        images = Gs.run(z, label, **Gs_kwargs) # [minibatch, height, width, channel]
        PIL.Image.fromarray(images[0], 'RGB').save(f'{outdir}/seed{seed:04d}.png')


In [ ]:
def valmap(value, istart, istop, ostart, ostop):
  return ostart + (ostop - ostart) * ((value - istart) / (istop - istart))

class OSN():
  min=-1
  max= 1

  def __init__(self,seed,diameter):
    self.tmp = OpenSimplex(seed)
    self.d = diameter
    self.x = 0
    self.y = 0

  def get_val(self,angle):
    self.xoff = valmap(np.cos(angle), -1, 1, self.x, self.x + self.d);
    self.yoff = valmap(np.sin(angle), -1, 1, self.y, self.y + self.d);
    return self.tmp.noise2d(self.xoff,self.yoff)

def get_noiseloop(endpoints, nf, d, start_seed):
    features = []
    zs = []
    for i in range(512):
      features.append(OSN(i+start_seed,d))

    inc = (np.pi*2)/nf
    for f in range(nf):
      z = np.random.randn(1, 512)
      for i in range(512):
        z[0,i] = features[i].get_val(inc*f) 
      zs.append(z)

    return zs
    
def line_interpolate(zs, steps):
   out = []
   for i in range(len(zs)-1):
    for index in range(steps):
     fraction = index/float(steps) 
     out.append(zs[i+1]*fraction + zs[i]*(1-fraction))
   return out
   
def generate_zs_from_seeds(seeds,Gs):
    zs = []
    for seed_idx, seed in enumerate(seeds):
        rnd = np.random.RandomState(seed)
        z = rnd.randn(1, *Gs.input_shape[1:]) # [minibatch, component]
        zs.append(z)
    return zs

def generate_latent_images(zs, truncation_psi, outdir, save_npy,prefix):
    Gs_kwargs = {
        'output_transform': dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True),
        'randomize_noise': False
    }
    
    if not isinstance(truncation_psi, list):
        truncation_psi = [truncation_psi] * len(zs)
    
    for z_idx, z in enumerate(zs):
        if isinstance(z,list):
          z = np.array(z).reshape(1,512)
        elif isinstance(z,np.ndarray):
          z.reshape(1,512)
        print('Generating image for step %d/%d ...' % (z_idx, len(zs)))
        Gs_kwargs['truncation_psi'] = truncation_psi[z_idx]
        noise_rnd = np.random.RandomState(1) # fix noise
        tflib.set_vars({var: noise_rnd.randn(*var.shape.as_list()) for var in noise_vars}) # [height, width]
        images = Gs.run(z, None, **Gs_kwargs) # [minibatch, height, width, channel]
        PIL.Image.fromarray(images[0], 'RGB').save(f'{outdir}/{prefix}{z_idx:05d}.png')
        if save_npy:
          np.save(dnnlib.make_run_dir_path('%s%05d.npy' % (prefix,z_idx)), z)

def generate_latent_walk(network_pkl, truncation_psi, outdir, walk_type, frames, seeds, npys, save_vector, diameter=2.0, start_seed=0 ):
    global _G, _D, Gs, noise_vars
    tflib.init_tf()
    print('Loading networks from "%s"...' % network_pkl)
    with dnnlib.util.open_url(network_pkl) as fp:
        _G, _D, Gs = pickle.load(fp)

    os.makedirs(outdir, exist_ok=True)
    
    # Render images for dlatents initialized from random seeds.
    Gs_kwargs = {
        'output_transform': dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True),
        'randomize_noise': False
    }
    if truncation_psi is not None:
        Gs_kwargs['truncation_psi'] = truncation_psi

    noise_vars = [var for name, var in Gs.components.synthesis.vars.items() if name.startswith('noise')]
    zs = []
    
    if(len(seeds) > 0):
        zs = generate_zs_from_seeds(seeds,Gs)
    # elif(len(npys) > 0):
    #     zs = npys
        
    if(len(zs) > 2 ):
        print('not enough values to generate walk')
#         return false;

    walk_type = walk_type.split('-')
    
    if walk_type[0] == 'line':
        number_of_steps = int(frames/(len(zs)-1))+1
    
        if (len(walk_type)>1 and walk_type[1] == 'w'):
          ws = []
          for i in range(len(zs)):
            ws.append(convertZtoW(zs[i]))
          points = line_interpolate(ws,number_of_steps)
          zpoints = line_interpolate(zs,number_of_steps)
        else:
          points = line_interpolate(zs,number_of_steps)

    # from Gene Kogan
    elif walk_type[0] == 'bspline':
        # bspline in w doesnt work yet
        # if (len(walk_type)>1 and walk_type[1] == 'w'):
        #   ws = []
        #   for i in range(len(zs)):
        #     ws.append(convertZtoW(zs[i]))

        #   print(ws[0].shape)
        #   w = []
        #   for i in range(len(ws)):
        #     w.append(np.asarray(ws[i]).reshape(512,18))
        #   points = get_latent_interpolation_bspline(ws,frames,3, 20, shuffle=False)
        # else:
          z = []
          for i in range(len(zs)):
            z.append(np.asarray(zs[i]).reshape(512))
          points = get_latent_interpolation_bspline(z,frames,3, 20, shuffle=False)

    # from Dan Shiffman: https://editor.p5js.org/dvs/sketches/Gb0xavYAR
    elif walk_type[0] == 'noiseloop':
        points = get_noiseloop(None,frames,diameter,start_seed)

    if (walk_type[0] == 'line' and len(walk_type)>1 and walk_type[1] == 'w'):
      # print(points[0][:,:,1])
      # print(zpoints[0][:,1])
      # ws = []
      # for i in enumerate(len(points)):
      #   ws.append(convertZtoW(points[i]))
      generate_images_in_w_space(points, truncation_psi,save_vector,'frame')
    elif (len(walk_type)>1 and walk_type[1] == 'w'):
      print('%s is not currently supported in w space, please change your interpolation type' % (walk_type[0]))
    else:
      generate_latent_images(points, truncation_psi, outdir, save_vector,'frame')

In [ ]:
rm -r /content/out

In [ ]:
generate_latent_walk("/content/ladiescrop.pkl", .7, "/content/out/", "noiseloop", 1440, [20,50,20], None, False, diameter=2.0, start_seed=100)

Loading networks from "/content/ladiescrop.pkl"...
not enough values to generate walk
Generating image for step 0/1440 ...
Generating image for step 1/1440 ...
Generating image for step 2/1440 ...
Generating image for step 3/1440 ...
Generating image for step 4/1440 ...
Generating image for step 5/1440 ...
Generating image for step 6/1440 ...
Generating image for step 7/1440 ...
Generating image for step 8/1440 ...
Generating image for step 9/1440 ...
Generating image for step 10/1440 ...
Generating image for step 11/1440 ...
Generating image for step 12/1440 ...
Generating image for step 13/1440 ...
Generating image for step 14/1440 ...
Generating image for step 15/1440 ...
Generating image for step 16/1440 ...
Generating image for step 17/1440 ...
Generating image for step 18/1440 ...
Generating image for step 19/1440 ...
Generating image for step 20/1440 ...
Generating image for step 21/1440 ...
Generating image for step 22/1440 ...
Generating image for step 23/1440 ...
Generating i

In [ ]:
#!ffmpeg -r 24 -i /content/out/frame%05d.png -vf scale=iw*.6:ih -vcodec libx264 -pix_fmt yuv420p /content/interpolation-shrink.mp4
!ffmpeg -r 24 -i /content/out/frame%05d.png -vcodec libx264 -pix_fmt yuv420p /content/noiseloop3.mp4

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib